<a href="https://colab.research.google.com/github/kushal2022/Pytorch/blob/main/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Basic PyTorch

In [ ]:
import torch

weights = torch.tensor([0.2126, 0.7152, 0.0722], requires_grad=True)

for epoch in range(10):
  model_output = (weights * 3).sum()

  model_output.backward()

  print(weights.grad)

  weights.grad.zero_()

tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])


In [ ]:
weights = torch.tensor([0.2126, 0.7152, 0.0722], requires_grad=True)

# Wrap the weights tensor in a list to make it an iterable
optimizer = torch.optim.SGD([weights], lr=0.01)

for epoch in range(10):
  model_output = (weights * 3).sum()

  model_output.backward()

  print(weights.grad)

  optimizer.step()

  optimizer.zero_grad()

tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])
tensor([3., 3., 3.])


**Back propagation example with pytorch**

In [ ]:
x = torch.tensor(1.0)
y = torch.tensor(2.0)

w = torch.tensor(1.0, requires_grad=True)

# forward pass and compute the loss
y_hat = w * x
loss = (y_hat - y)**2

print(loss)

# backward pass
loss.backward()
print(w.grad)

### update weights
### next forward and backward

tensor(1., grad_fn=<PowBackward0>)
tensor(-2.)


In [3]:
from sklearn.preprocessing import OneHotEncoder

data = [['Red'], ['Green'], ['Blue'], ['Red'], ['Yellow'], ['Monkey'], ['Monkey']]

#creating and applying OneHotEncoder
encoder = OneHotEncoder()
encoder.fit(data)
encoded_data = encoder.transform(data).toarray()

print(encoded_data)

[[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [8]:
import torch
import torch.nn as nn
import numpy as np

def cross_entropy(actual_class, predicted_probabilities):
  loss = -np.sum(actual_class * np.log(predicted_probabilities))
  return loss

Y = np.array([1, 0, 0])

#y_pred has probabilities
Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)
print(f'Loss1 numpy: {l1:.4f}')
print(f'Loss2 numpy: {l2:.4f}')

Loss1 numpy: 0.3567
Loss2 numpy: 2.3026
